In [1]:
import requests
import sqlite3
import pandas as pd
import logging
import numpy as np
from bs4 import BeautifulSoup
import datetime

In [2]:
url = "https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks"

In [3]:
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = '/home/project/Countries_by_GDP.csv'
df = pd.DataFrame(columns=["Name","MC_USD_Billion"])
count = 0
log_file="code_log.txt"

In [11]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries

def log_progress(log_file,log_text):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a") as log_file:
        log_file.write(f"{timestamp} : {log_text}\n")

def extract(url, df):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    html_page = requests.get(url).text
    data = BeautifulSoup(html_page, 'html.parser')
    banks_data = data.find_all('tbody')
    rows=banks_data[0].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)>2:
            dict_data = {
                "Name" : col[1].text.strip(),
                "MC_USD_Billion" : col[2].text.strip()
            }
            df1 = pd.DataFrame(dict_data, index=[0])
            df = pd.concat([df,df1],ignore_index=True)
    return df
    

def transform(data_f, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
#     print(data_f,"data")
    fr_d = pd.read_csv(csv_path)
#     print(fr_d,"cvs read")
    rate_dict = dict(zip(fr_d["Currency"], fr_d["Rate"]))
#     print(rate_dict,"dict")
    data_f['MC_GBP_Billion'] = [np.round(float(x)*rate_dict['GBP'],2) for x in data_f['MC_USD_Billion']]
    data_f["MC_INR_Billion"] = [np.round(float(x)*rate_dict['INR'],2) for x in data_f['MC_USD_Billion']]
    data_f["MC_EUR_Billion"] = [np.round(float(x)*rate_dict['EUR'],2) for x in data_f['MC_USD_Billion']]

    return data_f

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    data_f.to_csv("Largest_banks_data.csv")

def load_to_db(df, connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name,connection,if_exists='replace', index=False)  # Use 'replace' to overwrite the table if it already exists
    cursor = connection.cursor()
#     cursor.execute("Select * from Largest_banks")
#     rows = cursor.fetchall()
#     print(rows)

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    cursor=sql_connection.cursor()
    cursor.execute(query_statement)
    rows = cursor.fetchall()
    print(rows)

    ''' Here, you define the required entities and call the relevant
    functions in the correct order to complete the project. Note that this
    portion is not inside any function.'''

In [14]:
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = '/home/project/Countries_by_GDP.csv'
df = pd.DataFrame(columns=["Name","MC_USD_Billion"])
count = 0
log_file="code_log.txt"

log_progress(log_file,"Preliminaries complete. Initiating ETL process")

data_f = extract(url,df)
log_progress(log_file,"Data extraction complete. Initiating Transformation process")
print("data extracted\n",data_f)

u = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
df_trans = transform(data_f,u)
print("data transformed\n",df_trans)
log_progress(log_file,"Data transformation complete. Initiating Loading process")

path = "./Largest_banks_data.csv"
load_to_csv(df_trans,path)
log_progress(log_file,"Data saved to CSV file")

log_progress(log_file,"SQL Connection initiated")
connection = sqlite3.connect('Banks.db')
load_to_db(df_trans,connection,table_name)
log_progress(log_file,"Process Complete")

run_query("SELECT * FROM Largest_banks", connection)
connection.close()
log_progress(log_file,"Server Connection closed")

data extracted
                                       Name MC_USD_Billion
0                           JPMorgan Chase         432.92
1                          Bank of America         231.52
2  Industrial and Commercial Bank of China         194.56
3               Agricultural Bank of China         160.68
4                                HDFC Bank         157.91
5                              Wells Fargo         155.87
6                        HSBC Holdings PLC         148.90
7                           Morgan Stanley         140.83
8                  China Construction Bank         139.82
9                            Bank of China         136.81
                                      Name MC_USD_Billion
0                           JPMorgan Chase         432.92
1                          Bank of America         231.52
2  Industrial and Commercial Bank of China         194.56
3               Agricultural Bank of China         160.68
4                                HDFC Bank         157.9

In [15]:
with open(log_file, "r") as log_file:
        text=log_file.read()
        print(text)


2025-03-11 15:05:09 : Initiated web scraping
2025-03-11 15:26:59 : Preliminaries complete. Initiating ETL process
2025-03-11 15:27:02 : Data extraction complete. Initiating Transformation process
2025-03-11 15:27:02 : Data transformation complete. Initiating Loading process
2025-03-11 15:27:02 : Data saved to CSV file
2025-03-11 15:27:02 : SQL Connection initiated
2025-03-11 15:27:02 : Process Complete
2025-03-11 15:27:02 : Server Connection closed
2025-03-11 16:00:05 : Preliminaries complete. Initiating ETL process
2025-03-11 16:00:06 : Data extraction complete. Initiating Transformation process
2025-03-11 16:00:07 : Data transformation complete. Initiating Loading process
2025-03-11 16:00:07 : Data saved to CSV file
2025-03-11 16:00:07 : SQL Connection initiated
2025-03-11 16:00:32 : Preliminaries complete. Initiating ETL process
2025-03-11 16:00:34 : Data extraction complete. Initiating Transformation process
2025-03-11 16:00:34 : Data transformation complete. Initiating Loading pro